In [1]:
con.close()

NameError: name 'con' is not defined

In [7]:
import pandas as pd
import duckdb
from sklearn.model_selection import train_test_split
from fastai.tabular.all import *
import matplotlib.pyplot as plt
import numpy as np
from ai_functions import model_queries as mq

# Connect to database
con = duckdb.connect("data/deadlock.db")

# Execute your SQL query to get the structured data
df = mq.fetch_training_data(con)

if df is None or len(df) == 0:
    print("Failed to retrieve training data")
    sys.exit()

# Check the data
print(f"Dataset shape: {df.shape}")
print(f"Sample of data:\n{df.head()}")

#check balance of the target variable
print(f"Target distribution:\n{df['team0_won'].value_counts(normalize=True)}")

AttributeError: module 'ai_functions.model_queries' has no attribute 'fetch_training_data'

In [ ]:
# Remove non-feature columns
feature_df = training_data.drop(['match_id', 'start_time', 'winning_team'], axis=1)

# Define target variable
target = 'team0_won'

# Split into features and target
X = feature_df.drop(target, axis=1)
y = feature_df[target]

# Split into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Create FastAI TabularPandas object
procs = [Categorify, FillMissing, Normalize]
splits = (list(range(len(X_train))), list(range(len(X_train), len(X_train) + len(X_valid))))
to = TabularPandas(pd.concat([X_train, X_valid]), procs, 
                   cat_names=[], 
                   cont_names=list(X_train.columns),
                   y_names=target, 
                   splits=splits)

# Create DataLoaders
dls = to.dataloaders(bs=64)

In [ ]:
# Create and train the random forest model
learn = tabular_learner(dls, metrics=accuracy)

# Find optimal learning rate
learn.lr_find()

# Train the model
learn.fit_one_cycle(5, 1e-2)

# Save the model
learn.export('deadlock_match_prediction_model.pkl')

In [ ]:
# Evaluate model performance
interp = ClassificationInterpretation.from_learner(learn)

# Confusion matrix
interp.plot_confusion_matrix()

# Feature importance
importance = interp.feature_importance()
plt.figure(figsize=(12, 8))
plt.barh(X.columns[importance[1]], importance[0])
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.tight_layout()
plt.show()

# Top losses (most incorrect predictions)
interp.plot_top_losses(10)